# ДЗ 2 Гладышев В.В.¶

In [1]:
import pandas as pd

Наши новости

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [4]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [6]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

C:\Users\Лариса\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
pip install razdel

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [9]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Лариса\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [12]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [13]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [14]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-13-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 28.4 s


In [15]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 3min 49s


А теперь в 3 строчки обучим нашу модель

In [16]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [17]:
common_dictionary[10]

'ватутин'

Все просто - это словарь наших слов

Запускаем обучение

In [18]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 33.7 s


In [19]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [20]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(3, 0.62384045), (11, 0.2107564), (21, 0.1456734)]

In [21]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: продукция двигатель майкл профессия клиника тесный остановка
topic_1: человек который женщина земля тело мужчина nn
topic_2: сша россия правительство президент закон который санкция
topic_3: это всё мочь человек который риск граница
topic_4: рейс гражданство вдвое консультация маловероятный парижский дорого
topic_5: наука игра девочка научный лауреат компьютерный хороший
topic_6: банк гражданин млрд федеральный депутат ребёнок фронт
topic_7: газ турецкий турция полоса сон египетский выручка
topic_8: который nn год это город человек также
topic_9: британский солнце великобритания лондон медик nthe британец
topic_10: рубль рак офицер армия мвд транспорт дождь
topic_11: тур бензин соцсеть испания умирать спрос антонов
topic_12: век управление обращение включать nn диапазон маршрут
topic_13: сократиться писать пилотировать музыка мэй альянс студия
topic_14: год который это новый также компания свой
topic_15: космос атмосферный остаток разбираться таиланд индонезия китаец
topic_16:

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [22]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [23]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.158281,0.000000,0.0,0.000000,0.705294,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,4896,0.0,0.264349,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.272409,0.0,0.044359,0.0
2,4897,0.0,0.000000,0.000000,0.623722,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.145801,0.0,0.000000,0.0
3,4898,0.0,0.000000,0.000000,0.623989,0.0,0.038409,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.231898,0.0,0.000000,0.0
4,4899,0.0,0.000000,0.000000,0.000000,0.0,0.527861,0.415099,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [24]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [25]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, 
                    topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [26]:
doc_dict[293622]

array([0.03530033, 0.15058766, 0.25277495, 0.08165907, 0.        ,
       0.11838121, 0.        , 0.        , 0.20360544, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.14675346,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [27]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, fnc=np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = fnc(user_vector, 0)
    return user_vector

In [28]:
get_user_embedding(user_articles_list)

array([0.00194522, 0.0570304 , 0.19797611, 0.02437164, 0.00305446,
       0.        , 0.02472957, 0.        , 0.05313613, 0.        ,
       0.00700539, 0.        , 0.00883291, 0.        , 0.15957507,
       0.        , 0.00949442, 0.04287643, 0.00587541, 0.28007621,
       0.        , 0.01000405, 0.00400511, 0.09585895, 0.00288409])

Интересовался новостями с топиками topic_3, topic_14 (что-то про политику и государство)

In [29]:
#users['articles'].iloc[33]

In [30]:
#" ".join(news[news['doc_id']==323186]['title'].iloc[0])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

Загрузим нашу разметку

In [31]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, \
                            roc_auc_score, \
                            precision_score, \
                            classification_report, \
                            precision_recall_curve, \
                            confusion_matrix

%matplotlib inline

### Рассчитаем Precision, Recall, F_score

In [36]:
agfunc_list = [np.max, np.median, np.mean]
agfunc_name = ["max", "median", "mean"]
pred_dict = {}

for i in range(3):
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(
                        lambda x: get_user_embedding(x, agfunc_list[i]), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+
                        ['topic_{}'.format(i) for i in range(25)]]
    user_embeddings = pd.merge(user_embeddings, target, 'left')
    #разделим данные на train/test
    X_train, \
    X_test, \
    y_train, \
    y_test = train_test_split(user_embeddings[[
                        'topic_{}'.format(i) for i in range(25)]], 
                        user_embeddings['churn'], random_state=0)
    logreg = LogisticRegression()
    #обучим 
    logreg.fit(X_train, y_train)
    pred_dict[agfunc_name[i]] = logreg.predict_proba(X_test)[:, 1]
    # Рассчитаем Precision, Recall, F_score
    precision, recall, thresholds = precision_recall_curve(y_test, 
                                                           pred_dict[agfunc_name[i]].tolist())
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print(f'Функция агрегации {agfunc_name[i]} \n'
          f'\tЛучший Threshold: {thresholds[ix]:.3f}, '
          f'F-Score: {fscore[ix]:.3f}, '
          f'Precision: {precision[ix]:.3f}, '
          f'Recall {recall[ix]:.3f} \n'
          f'\tROC_AUC {roc_auc_score(y_test, pred_dict[agfunc_name[i]].tolist()):.3f}\n')

Функция агрегации max 
	Лучший Threshold: 0.364, F-Score: 0.767, Precision: 0.762, Recall 0.771 
	ROC_AUC 0.963

Функция агрегации median 
	Лучший Threshold: 0.287, F-Score: 0.729, Precision: 0.712, Recall 0.747 
	ROC_AUC 0.958

Функция агрегации mean 
	Лучший Threshold: 0.237, F-Score: 0.633, Precision: 0.540, Recall 0.763 
	ROC_AUC 0.930



### Выводы
Агрегирующая функция Max не размывает пики в распределении (в отличие от mean), а подчёркивает их